## Gradient Boosting Hyperparameter Tuning (Recall)

In [52]:
# Load Libraries

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import recall_score

from sklearn.ensemble import GradientBoostingClassifier

import optuna

In [54]:
# Load Data

df = pd.read_csv('bank_4.csv', index_col=0)

In [56]:
# Train / Test Split

X = df.drop(columns=['churn', 'complain', 'umap_1', 'umap_2'])
y = df['churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [58]:
# Hyperparameter tuning

def objective(trial):

    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 2000),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 1.0),
        'max_depth': trial.suggest_int('max_depth', 2, 50),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 32),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 32)
    }

    model = GradientBoostingClassifier(
        **params,
        random_state=42)
    
    threshold = 0.33
    
    skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
    scores = []
    
    for tr, te in skf.split(X_train, y_train):
        
        X_tr, X_te = X_train.iloc[tr], X_train.iloc[te]
        y_tr, y_te = y_train.iloc[tr], y_train.iloc[te]
        
        model.fit(X_tr, y_tr)
        prob = model.predict_proba(X_te)[:, 1]
        y_pred = np.where(prob < threshold, 0, 1)
        
        scores.append(recall_score(y_te, y_pred))
        
    return np.mean(scores)

In [60]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2024-07-08 18:49:02,438] A new study created in memory with name: no-name-38b5e900-59ae-4f15-bfd9-81009c212a2c


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2024-07-08 18:50:14,432] Trial 0 finished with value: 0.4711656441717791 and parameters: {'n_estimators': 1804, 'learning_rate': 0.1282261686622636, 'max_depth': 38, 'min_samples_split': 18, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.4711656441717791.
[I 2024-07-08 18:50:32,947] Trial 1 finished with value: 0.4840490797546012 and parameters: {'n_estimators': 779, 'learning_rate': 0.633366369967308, 'max_depth': 22, 'min_samples_split': 7, 'min_samples_leaf': 15}. Best is trial 1 with value: 0.4840490797546012.
[I 2024-07-08 18:51:07,969] Trial 2 finished with value: 0.4815950920245398 and parameters: {'n_estimators': 1357, 'learning_rate': 0.3098100872549122, 'max_depth': 33, 'min_samples_split': 5, 'min_samples_leaf': 13}. Best is trial 1 with value: 0.4840490797546012.
[I 2024-07-08 18:51:57,693] Trial 3 finished with value: 0.5895705521472393 and parameters: {'n_estimators': 1843, 'learning_rate': 0.12646497515922503, 'max_depth': 3, 'min_samples_split': 15, 'min_samp

In [66]:
print("Best trial:", study.best_trial)
print("Best hyperparameters:", study.best_params)

Best trial: FrozenTrial(number=81, state=1, values=[0.6122699386503069], datetime_start=datetime.datetime(2024, 7, 8, 20, 54, 28, 569279), datetime_complete=datetime.datetime(2024, 7, 8, 20, 55, 57, 365007), params={'n_estimators': 1888, 'learning_rate': 0.0031180517539165756, 'max_depth': 6, 'min_samples_split': 14, 'min_samples_leaf': 23}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=2000, log=False, low=100, step=1), 'learning_rate': FloatDistribution(high=1.0, log=False, low=0.001, step=None), 'max_depth': IntDistribution(high=50, log=False, low=2, step=1), 'min_samples_split': IntDistribution(high=32, log=False, low=2, step=1), 'min_samples_leaf': IntDistribution(high=32, log=False, low=1, step=1)}, trial_id=81, value=None)
Best hyperparameters: {'n_estimators': 1888, 'learning_rate': 0.0031180517539165756, 'max_depth': 6, 'min_samples_split': 14, 'min_samples_leaf': 23}
